In [ ]:
import torch
import pickle
from uncertainty_rnn import BoardGuesserNet
import torch.optim as optim
import numpy as np
import time


# Current emission Encodding and Input to Network:
    Channels 1-12: stores result of sensing. emission_matrix[0, 2, 2] means there is a rook at (2, 2)

    Channel 13: emission_matrix[12, 1, 1] == 1 means that the opponent took a piece at this location

    Channel 14: move requested from
    Channel 15: move requested to

    Channel 16: move actually taken from
    Channel 17: move actually take to

    Channel 18: if emission_matrix[17, 2, 2] == 1, it means the opponent took a piece at 2, 2

    Channel 19: black or white. 
   

# What is the essential idea of using an RNN?

### You feed in emissions (observations) into the network and it outputs the best hypothesis for the state of the board.

### Why does this work? Hidden states convey past information about state history
    
# When do we query the RNN?

## Choose Sense
    We query the RNN here because we need to choose the sense that resolves the most doubt in our network (Ive got to still work on this). For now we should just randomly choose senses

## Choose Move
    We query here because our policy network needs a good estimate of the state of the board
    

# IMPORTANT NOTE: because we are querying the network twice per turn, we have two datapoints to train on per turn.

# To see how data is collected please look at random_agent_save_obs.py

# TODO
    -create network for online inference (will read in weights of trained network)
     

    

In [5]:
# what if we deal with uncertainty with an RNN?
#Treat this like any other machine learning problem X -> Y, except X is obs and Y is truth board

#first we collect data
start = time.time()
!python record_data.py random_agent_save_obs random_agent_save_obs
print("total time to run game", time.time() - start)

Game Over!
BLACK won by king capture.
total time to run game 0.37033867835998535


In [7]:
# now lets load the data in
in_ = open("white_game_obs.pkl", "rb")
white_sense_list, white_truth_board_list = pickle.load(in_)
assert not np.all(white_truth_board_list[0] == white_truth_board_list[-1])
in_.close()

In [ ]:
# white sense list is basically a list of one hot encoding of sensing data
print("number of turns in game", white_sense_list[0].shape)
print("dimension of a sense", white_sense_list[0].shape)
# 1st dimension is number of turns, second dimension

In [5]:
#load the recurrent network
guessNet = BoardGuesserNet()

In [6]:
#set up loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.SGD(guessNet.parameters(), lr=0.001, momentum=0.9)


In [7]:


start_train_batch = time.time()
# prepare data for training
X_train_batch = torch.Tensor(white_sense_list)
y_train_batch = torch.Tensor(white_truth_board_list)
pred_labels = guessNet(X_train_batch)
loss = criterion(pred_labels, y_train_batch)

loss.backward()
optimizer.step() # magic
print("time to train batch", time.time() - start_train_batch)

conv 1 out shape torch.Size([46, 32, 6, 6])
conv 2 out shape torch.Size([46, 64, 5, 5])
pool_2_out shape torch.Size([46, 64, 2, 2])
time to train batch 0.07606863975524902
